In [1]:
%pylab inline
import glob
import pandas as pd
import math
import dateutil

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = '../csv/original/'
filenames = glob.glob(path + "/*.csv")

list_ = []
frame = pd.DataFrame()

# Para cada um dos csv
for filename in filenames:
    # Leia o CSV
    df = pd.read_csv(filename, index_col = False)
    # Coloque na lista de CSV
    list_.append(df)

frame = pd.concat(list_)
frame.columns

Index([u'Data Emissão', u'OB Origem', u'NE Origem', u'No Processo', u'Credor',
       u'Nome Credor', u'Finalidade', u'Fonte de Recurso',
       u'Natureza Detalhada', u'Nome Natureza Detalhada', u'UG Executora',
       u'Nome UG Executora', u'Mês Emissão', u'Pago'],
      dtype='object')

In [3]:
df_clean = frame

In [4]:
# Limpeza do dataframe
df_clean['Pago'] = df_clean['Pago'].str.strip()
df_clean['Pago'] = df_clean['Pago'].str.replace(' ','')
df_clean['Pago'] = df_clean['Pago'].str.replace('(','')
df_clean['Pago'] = df_clean['Pago'].str.replace(')','')
df_clean['Pago'] = df_clean['Pago'].str.replace('.','')
df_clean['Pago'] = df_clean['Pago'].str.replace(',','.')

df_clean['Nome Credor'] = df_clean['Nome Credor'].str.replace('\n',' ')
df_clean['Nome Credor'] = df_clean['Nome Credor'].str.replace('\r',' ')

df_clean['Nome UG Executora'] = df_clean['Nome UG Executora'].str.replace('\n',' ')
df_clean['Nome UG Executora'] = df_clean['Nome UG Executora'].str.replace('\r',' ')

df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('\n',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('\r',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('.',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('-',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace(',',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('/',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('  ',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('   ',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('    ',' ')
df_clean['Nome Natureza Detalhada'] = df_clean['Nome Natureza Detalhada'].str.replace('T I','TI')

df_clean['Finalidade'] = df_clean['Finalidade'].str.replace('\n',' ')
df_clean['Finalidade'] = df_clean['Finalidade'].str.replace('//','')
df_clean['Finalidade'] = df_clean['Finalidade'].str.replace('\r',' ')

In [5]:
df_clean.shape

(103465, 14)

In [6]:
# Removendo o que? Não estamos perdendo linhas importântes?
df_clean = df_clean.dropna()

# Tem como converter esses 'NaN' para 0 ?

# Como usar o astype(int) ou astype(float) sem dropar esses valores acima?

In [7]:
df_clean.shape

# O que tinha nas 20 mil linhas que foram dropadas?

(83431, 14)

### EXEMPLO DO PROBLEMA QUE É DROPAR AS LINHAS QUE TEM NaN

In [8]:
# Exemplo

exemplo = pd.DataFrame({"Descrição": ['Serviços', 'Reformas', 'Gasolina'],
"Setor": [np.nan, np.nan, 'Transporte'], "Valor Gasto": [1000, 2000, 3000]})
exemplo

,Descrição,Setor,Valor Gasto
0,Serviços,NaN,1000
1,Reformas,NaN,2000
2,Gasolina,Transporte,3000


In [9]:
# Se eu dropar as linhas que tiver NaN perco 2 linhas com valor de gastos importantes

exemplo = exemplo.dropna()
exemplo

,Descrição,Setor,Valor Gasto
2,Gasolina,Transporte,3000


### Fim do Exemplo

In [10]:
# Como tratar esse erro?

# Convertendo uma string para inteiro
df_clean['Mês Emissão'] = df_clean['Mês Emissão'].astype(int)

# Convertendo uma string para float
df_clean['Pago'] = df_clean['Pago'].astype(float)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [11]:
# Quantidade de lançamentos de gastos de cada mês
df_clean['Mês Emissão'].value_counts()

11    20188
5      7819
4      6825
2      6736
3      6582
7      6548
9      5871
8      5609
10     5510
12     4569
6      4316
1      2858
Name: Mês Emissão, dtype: int64

In [12]:
# Agrupando e somando as linhas com descrição iguais
table_gastos = df_clean.groupby('Nome Natureza Detalhada')['Pago'].sum()

# Configurando modo de exibição de floats
pd.options.display.float_format = '${:,.2f}'.format

In [13]:
# Convertendo Series para Dataframe
df_gastos = pd.DataFrame(table_gastos).reset_index()

# Ordenando em ordem alfabética para facilitar visualização de descrições que podem ser agrupadas
df_gastos = df_gastos.sort_values(by='Nome Natureza Detalhada')


df_gastos.head()
# Repare que algumas linhas da natureza detalhada possuem descrições diferêntes,
# porém deveriam está agrupadas juntas (ex.: 13 sal)

,Nome Natureza Detalhada,Pago
0,13 SALARIO,"$62,074,505.06"
1,13 SALARIO PENSOES CIVIS,"$4,598,805.52"
2,13 SALARIO PESSOAL CIVIL,"$26,220,733.49"
3,ABONO DE PERMANENCIA,"$9,066,114.39"
4,ADICIONAL DE INSALUBRIDADE,"$7,878,905.06"


In [14]:
# Criando uma Series com a coluna "Nome Natureza Detalhada"
names = df_gastos['Nome Natureza Detalhada']

# Função para achar prefixo em comum das frases e substituir na original
# ex.: 13 salario pessoal civil -> 13 salario
def find_prefix(x):
    for name in names:
        if len(name) >= len(x):
            continue
        if name == x[:len(name)]:
            return name
    return x

# Aplicando a função find_prefix para todos elementos da coluna
df_gastos['Nome Natureza Detalhada'] = df_gastos['Nome Natureza Detalhada'].apply(find_prefix)

# Resultado
df_gastos.head()

,Nome Natureza Detalhada,Pago
0,13 SALARIO,"$62,074,505.06"
1,13 SALARIO,"$4,598,805.52"
2,13 SALARIO,"$26,220,733.49"
3,ABONO DE PERMANENCIA,"$9,066,114.39"
4,ADICIONAL DE INSALUBRIDADE,"$7,878,905.06"


In [ ]:
# Esta função aplica o conceito da Distância Levenshtein em todos os elementos
# e muda a descrição cuja a Distância Levenshtein seja menor ou igual a 3
from nltk.metrics import edit_distance
names = df_gastos['Nome Natureza Detalhada']

def find_similar(x):
    for name in names:
        # Esta função manipula as duas string do parâmetro e calcula a quantidade de operações para
        # transformar name em x
        if edit_distance(name, x) <= 3:
            return name
    return x

df_gastos['Nome Natureza Detalhada'] = df_gastos['Nome Natureza Detalhada'].apply(find_similar)
df_gastos.head()

In [ ]:
# Agrupando e somando as linhas com descrição iguais
df_gastos = df_gastos.reset_index()
df_gastos = df_gastos.groupby('Nome Natureza Detalhada')['Pago'].sum()
df_gastos

In [ ]:
# Salvando o resultado em um csv
df_gastos.to_csv('../csv/parsed/Natureza_Detalhada_X_Pago.csv')